## Question 9: Distance Matrix Calculation

In [1]:
import pandas as pd
df = pd.read_csv('dataset-2.csv')
df

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1
5,1001410,1001412,15.6
6,1001412,1001414,18.2
7,1001414,1001416,13.2
8,1001416,1001418,13.6
9,1001418,1001420,12.9


In [33]:
import pandas as pd
import numpy as np

def calculate_distance_matrix(csv_file):

    required_columns = {'id_start', 'id_end', 'distance'}
    if not required_columns.issubset(df.columns):
        raise ValueError("CSV must contain 'id_start', 'id_end', and 'distance' columns")

    # Create a set of unique locations
    locations = set(df['id_start']).union(set(df['id_end']))
    locations = list(locations)
    
    # Initialize a distance matrix with NaN
    distance_matrix = pd.DataFrame(index=locations, columns=locations)
    distance_matrix = distance_matrix.fillna(np.inf) 
    # Populate the distance matrix with the known distances
    for index, row in df.iterrows():
        loc_a = row['id_start']
        loc_b = row['id_end']
        distance = row['distance']
        
        # Set the distance in both directions
        distance_matrix.loc[loc_a, loc_b] = distance
        distance_matrix.loc[loc_b, loc_a] = distance

    # Set diagonal values to 0
    np.fill_diagonal(distance_matrix.values, 0)
    
    # Calculate cumulative distances
    for k in locations:
        for i in locations:
            for j in locations:
                # Update distance if a shorter path is found
                if distance_matrix.loc[i, k] + distance_matrix.loc[k, j] < distance_matrix.loc[i, j]:
                    distance_matrix.loc[i, j] = distance_matrix.loc[i, k] + distance_matrix.loc[k, j]

    distance_matrix.replace(np.inf, np.nan, inplace=True)
    print("Distance Matrix:")
    print(distance_matrix)

    return distance_matrix

if __name__ == "__main__":
    distance_df = calculate_distance_matrix('dataset-2.csv')
    print("Returned Distance DataFrame:")
    print(distance_df) 


Distance Matrix:
         1001472  1001488  1004354  1004355  1004356  1001400  1001402  \
1001472     32.0    179.8    173.8    171.8    175.8    444.3    434.6   
1001488    179.8      8.0      6.0      8.0      4.0    264.5    254.8   
1004354    173.8      6.0      4.0      2.0      2.0    270.5    260.8   
1004355    171.8      8.0      2.0      1.4      4.0    272.5    262.8   
1004356    175.8      4.0      2.0      4.0      4.0    268.5    258.8   
1001400    444.3    264.5    270.5    272.5    268.5     19.4      9.7   
1001402    434.6    254.8    260.8    262.8    258.8      9.7     19.4   
1001404    414.4    234.6    240.6    242.6    238.6     29.9     20.2   
1001406    398.4    218.6    224.6    226.6    222.6     45.9     36.2   
1001408    376.7    196.9    202.9    204.9    200.9     67.6     57.9   
1001410    365.6    185.8    191.8    193.8    189.8     78.7     69.0   
1001412    350.0    170.2    176.2    178.2    174.2     94.3     84.6   
1001414    331.8    1

## Question 10: Unroll Distance Matrix

In [16]:
def unroll_distance_matrix(distance_matrix):
    unrolled_data = []

    # Get the list of locations from the distance matrix
    locations = distance_matrix.index

    # Iterate through the distance matrix
    for i in locations:
        for j in locations:
                distance = distance_matrix.loc[i, j]
                if not pd.isna(distance):
                    unrolled_data.append({'id_start': i, 'id_end': j, 'distance': distance})

    unrolled_df = pd.DataFrame(unrolled_data)

    return unrolled_df

In [19]:
unrolled_df = unroll_distance_matrix(distance_matrix)
print("Unrolled Distance DataFrame:")
print(unrolled_df)

Unrolled Distance DataFrame:
      id_start   id_end  distance
0      1001472  1001488     179.8
1      1001472  1004354     173.8
2      1001472  1004355     171.8
3      1001472  1004356     175.8
4      1001472  1001400     444.3
...        ...      ...       ...
1801   1001470  1001461      61.6
1802   1001470  1001462      56.5
1803   1001470  1001464      29.8
1804   1001470  1001466      21.3
1805   1001470  1001468      10.6

[1806 rows x 3 columns]


## Question 11: Finding IDs within Percentage Threshold

In [25]:
import pandas as pd
import numpy as np

def find_ids_within_ten_percentage_threshold(df, reference_id):
    # Calculate the average distance for the given reference_id
    average_distance = df[df['id_start'] == reference_id]['distance'].mean()
    
    # Check if average_distance is NaN (i.e., reference_id not found)
    if pd.isna(average_distance):
        return []

    # Calculate the 10% threshold values
    lower_bound = average_distance * 0.9
    upper_bound = average_distance * 1.1

    # Filter the DataFrame for id_start values within the threshold
    filtered_ids = df[(df['distance'] >= lower_bound) & (df['distance'] <= upper_bound)]
    result_ids = filtered_ids['id_start'].unique()
    result_ids_sorted = sorted(result_ids)

    return result_ids_sorted


In [28]:
reference_id = 1001472 
ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
print("IDs within 10% of average distance for reference ID {}: {}".format(reference_id, ids_within_threshold))


IDs within 10% of average distance for reference ID 1001472: [1001400, 1001402, 1001404, 1001406, 1001408, 1001410, 1001412, 1001414, 1001416, 1001418, 1001420, 1001422, 1001424, 1001426, 1001428, 1001430, 1001432, 1001434, 1001436, 1001437, 1001438, 1001440, 1001442, 1001444, 1001446, 1001448, 1001450, 1001452, 1001454, 1001456, 1001458, 1001460, 1001461, 1001462, 1001464, 1001466, 1001468, 1001470, 1001472, 1001488, 1004354, 1004355, 1004356]


## Question 12: Calculate Toll Rate

In [41]:
import pandas as pd

def calculate_toll_rate(df):
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }

    for vehicle, coefficient in rate_coefficients.items():
        df[vehicle] = df['distance'] * coefficient

    return df

dj = pd.read_csv('dataset-2.csv')
# Calculate toll rates
toll_rates_df = pd.DataFrame(calculate_toll_rate(dj))

print("                DataFrame with Toll Rates:")
print(toll_rates_df)

                DataFrame with Toll Rates:
    id_start   id_end  distance   moto    car     rv    bus  truck
0    1001400  1001402       9.7   7.76  11.64  14.55  21.34  34.92
1    1001402  1001404      20.2  16.16  24.24  30.30  44.44  72.72
2    1001404  1001406      16.0  12.80  19.20  24.00  35.20  57.60
3    1001406  1001408      21.7  17.36  26.04  32.55  47.74  78.12
4    1001408  1001410      11.1   8.88  13.32  16.65  24.42  39.96
5    1001410  1001412      15.6  12.48  18.72  23.40  34.32  56.16
6    1001412  1001414      18.2  14.56  21.84  27.30  40.04  65.52
7    1001414  1001416      13.2  10.56  15.84  19.80  29.04  47.52
8    1001416  1001418      13.6  10.88  16.32  20.40  29.92  48.96
9    1001418  1001420      12.9  10.32  15.48  19.35  28.38  46.44
10   1001420  1001422       9.6   7.68  11.52  14.40  21.12  34.56
11   1001422  1001424      11.4   9.12  13.68  17.10  25.08  41.04
12   1001424  1001426      18.6  14.88  22.32  27.90  40.92  66.96
13   1001426  10014

## Question 13: Calculate Time-Based Toll Rates

In [59]:
a=pd.read_csv("toll_rates.csv")
a

,id_start,id_end,distance,moto,car,rv,bus,truck
0,1,2,10,8.0,12.0,15.0,22.0,36.0
1,2,3,20,16.0,24.0,30.0,44.0,72.0
2,3,1,30,24.0,36.0,45.0,66.0,108.0


In [63]:
import pandas as pd
from datetime import time

def calculate_time_based_toll_rates(df):
    discount_factors = {
        'weekday': {
            (time(0, 0), time(10, 0)): 0.8,   
            (time(10, 0), time(18, 0)): 1.2,  
            (time(18, 0), time(23, 59, 59)): 0.8,
        },
        'weekend': 0.7      
    }
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    new_rows = []

    # Iterate over each unique (id_start, id_end) pair
    for (id_start, id_end), group in df.groupby(['id_start', 'id_end']):
        distance = group['distance'].values[0] 
        
        # For each day of the week
        for day in days_of_week:
            if day in ['Saturday', 'Sunday']:
                factor = discount_factors['weekend']
                new_row = {
                    'id_start': id_start,
                    'id_end': id_end,
                    'start_day': day,
                    'start_time': time(0, 0),  # 12:00 AM
                    'end_day': day,
                    'end_time': time(23, 59, 59),  # 11:59:59 PM
                    'moto': distance * factor,
                    'car': distance * factor,
                    'rv': distance * factor,
                    'bus': distance * factor,
                    'truck': distance * factor,
                }
                new_rows.append(new_row)
            else:
                for time_range, factor in discount_factors['weekday'].items():
                    start_time, end_time = time_range
                    new_row = {
                        'id_start': id_start,
                        'id_end': id_end,
                        'start_day': day,
                        'start_time': start_time,
                        'end_day': day,
                        'end_time': end_time,
                        'moto': distance * factor,
                        'car': distance * factor,
                        'rv': distance * factor,
                        'bus': distance * factor,
                        'truck': distance * factor,
                    }
                    new_rows.append(new_row)

    new_df = pd.DataFrame(new_rows)
    return new_df

# Example usage
if __name__ == "__main__":
    example_data = {
        'id_start': [1, 1, 2],
        'id_end': [2, 3, 1],
        'distance': [10, 20, 30]  
    }
    toll_rates_df = pd.DataFrame(example_data)

    calculated_df1 = calculate_time_based_toll_rates(toll_rates_df)
    print("DataFrame with Time-Based Toll Rates:")
    print(calculated_df1)


DataFrame with Time-Based Toll Rates:
    id_start  id_end  start_day start_time    end_day  end_time  moto   car  \
0          1       2     Monday   00:00:00     Monday  10:00:00   8.0   8.0   
1          1       2     Monday   10:00:00     Monday  18:00:00  12.0  12.0   
2          1       2     Monday   18:00:00     Monday  23:59:59   8.0   8.0   
3          1       2    Tuesday   00:00:00    Tuesday  10:00:00   8.0   8.0   
4          1       2    Tuesday   10:00:00    Tuesday  18:00:00  12.0  12.0   
5          1       2    Tuesday   18:00:00    Tuesday  23:59:59   8.0   8.0   
6          1       2  Wednesday   00:00:00  Wednesday  10:00:00   8.0   8.0   
7          1       2  Wednesday   10:00:00  Wednesday  18:00:00  12.0  12.0   
8          1       2  Wednesday   18:00:00  Wednesday  23:59:59   8.0   8.0   
9          1       2   Thursday   00:00:00   Thursday  10:00:00   8.0   8.0   
10         1       2   Thursday   10:00:00   Thursday  18:00:00  12.0  12.0   
11         1  